Este notebook fue utilizado para la realización del Web Scrapping de la página Glassdoor

In [1]:
#importing libraries
from selenium import webdriver
import sys
sys.path.insert(0,'/usr/lib/chromedriver')
import time
from selenium.common.exceptions import NoSuchElementException , ElementClickInterceptedException
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml
import re
import pandas as pd
import json
import time

pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

El primer paso para obtener la información de cada oferta salarial es obtener los enlaces de cada puesto que se ofrece en glassdoor

En la página de glassdoor, al momento de hacer la busqueda de un término, se enlistas diversas opciones, regularmente 30 por página. Puede haber múltuples páginas de este estilo para un mismo término pero notamos que, depués de la 7ma página, estas siguen un patron: https://www.glassdoor.com.mx_IPXX.htm?includeNoSalaryJobs=true donde XX simboliza el número de la página, por lo que hacemos dos funciones para los casos con patrones (a partir de 7) y sin patrón (1-6)

In [2]:
#A partir de la función que tiene el patrón, obtenemos las restantes páginas, por lo que solo será necesario insertar un enlace de alguna de estas, por ejemplo:
#'https://www.glassdoor.com.mx/Empleo/méxico-ingeniero-de-datos-empleos-SRCH_IL.0,6_IN169_KO7,25_IP7.htm?includeNoSalaryJobs=true'
#Las siguientes dos funciones devolverán una lista

#Función para obtener los enlaces que tienen ciertos patrones en la página de glassdor
def ScrapeLinks(driver, start_pg, end_pg, pattern, previous_links = None):
    scr_links = []
    #En caso de que tengamos links previos, los agregamos
    if previous_links is not None:
        scr_links = previous_links
    br = driver
    url_head = 'https://www.glassdoor.com.mx'
    #Extraemos la primera parte del enlace para iterar crear los enlaces restantes.
    head_link = re.findall(r'(.+)_IP\d+', pattern)[0]
    for pg_num in range(start_pg, end_pg+1):
        url = head_link + '_IP' + str(pg_num) +' .htm?includeNoSalaryJobs=true'
        #Obtenemos el código fuente de la página
        br.get(url)
        br.implicitly_wait(10)
        soup = BeautifulSoup(br.page_source , 'lxml')
        print(f"Scraping Page Number : {pg_num}")
        #Extraemos las etiquetas
        for a_tag in soup.find_all('a' , href = True):
            #Y buscamos las que contienen los enlaces de los trabajos
            pattern = re.compile('/partner/jobListing')
            #Además, obtenemos el enlacec
            if pattern.match(str(a_tag['href'])):
                #Si tenemos el enlace, lo agregamos a nuestra lista
                scr_links.append(url_head + str(a_tag['href']))
        #Para evitar problemas de bloqueo de la IP, vamos a dar 20 segundos de respiro por cada 10 páginas:
        if((pg_num-start_pg)%(10) == 0):
            time.sleep(20)
    #Se pueden repetir los enlaces, así que obtenemos los únicos
    scr_links = list(set(scr_links))
    print("Scraping of links is completed")
    return scr_links

#Habra links para los cuales no necesitemos iterar:
def ScrapeLink(driver, page, previous_links = None):
    scr_links = []
    #En caso de que tengamos links previos, los agregamos
    if previous_links is not None:
        scr_links = previous_links
    br = driver
    url_head = 'https://www.glassdoor.com.mx'
    #Obtenemos el código fuente de la página
    print(f"Scraping web page")
    br.get(page)
    br.implicitly_wait(10)
    soup = BeautifulSoup(br.page_source , 'lxml')
    #Extraemos las etiquetas
    for a_tag in soup.find_all('a' , href = True):
        #Y buscamos las que contienen los enlaces de los trabajos
        pattern = re.compile('/partner/jobListing')
        #Además, obtenemos el enlacec
        if pattern.match(str(a_tag['href'])):
            #Si tenemos el enlace, lo agregamos a nuestra lista
            scr_links.append(url_head + str(a_tag['href']))
    #Se pueden repetir los enlaces, así que obtenemos los únicos
    scr_links = list(set(scr_links))
    print("Scraping of links is completed")
    return scr_links

Para solo pasar una lista con 7 enlaces (véase que no es necesario más que 7 enlaces ya que a patir del 7 se sigue un patron) se crea la siguiente función

In [ ]:
#Colocamos start_pg = None, end_pg = None en caso de no tener más de 6 links
#list_links puede ser un solo enlace
def get_link_jobs(driver, list_links, start_pg = None, end_pg = None):
    print("Comienza Web Scrapping de página individual")
    links_scrapped = []
    #Obtenemos los enlaces de la primera página
    if(isinstance(list_links, list)):
        links_scrapped = ScrapeLink(driver, list_links[0])
    else:
        links_scrapped = ScrapeLink(driver, list_links)
    #Si la lista tiene más de 1 link, tenemos que hacer varias veces el webScrapping. Validamos que sea una lista y no un link, el cual también será valido en el anterior caso
    if (len(list_links) > 1) & isinstance(list_links, list):
        for i in range(len(list_links)-1):
            if i != 5:#Es decir, cuando no estemos en el link 7
                print("Comienza Web Scrapping de página individual")
                links_scrapped = ScrapeLink(driver, list_links[i+1], links_scrapped)
            else:#Cuando estemos en el link 7
                time.sleep(30) #Daremos un respiro antes de minar más páginas
                print("Comienza Web Scrapping de páginas con patrón")
                links_scrapped = ScrapeLinks(driver, start_pg, end_pg,
                                             list_links[i+1], links_scrapped)
    print("Número de links: ", len(links_scrapped))
    #Para este punto, ya tenemos nuestra lista de links
    return(links_scrapped)

Al obtener los enlaces de cada puesto laboral, podemos acceder a su código fuente y obtener la información de la oferta piblicada, para esto adaptamos la siguiente función para obtener diversos rubros como lo son el título del trabajo, la compañia que publica la oferta, el estado y ciudad del empleo, el rango salarial (mínimo y máximo), la moneda en que el rango salarial se otorga, la descripción del trabajo, la industria, un ranking de la compañia, las fechas en que se publicaron las ofertas y hasta cuando son validas y, finalente, el tipo de trabajo que se ofrece (remoto, presencial, etc.)

In [ ]:
def ScrapeData(driver , links):
    br = driver
    data_df = pd.DataFrame(columns = ['Job_title' , 'Company' , 'State' , 'City' , 'Min_Salary' ,'Max_Salary' , 'unitText',
                                      "SalaryCurrency", 'Job_Desc' , 'Industry'  , 'Rating' , 'Date_Posted' , 'Valid_until' , 'Job_Type' ,
                                      ])
    start = time.time()
    print("Gathering Information")
    for job in range(len(links)):
        print(f'Extracting information about job : {job+1}')
        br.get(links[job])
        br.implicitly_wait(10)
        soup = BeautifulSoup(br.page_source , 'lxml')
        try:
            #The rest of the data is available in scrip = 'application/ld+json' i.e in  json file
            json_file = soup.find('script' , {'type':'application/ld+json'})
            op_json = json.loads(str(json_file.text) , strict = False)
            #title
            try:
                title = op_json['title']
                #print(title)
            except:
                title  = None
                print("except: title")
            #Dateposted
            try:
                post_date = op_json['datePosted']
            except:
                post_date = None
            #type
            try:
                job_type = op_json['employmentType']
            except:
                job_type = None
            #Annual Salary Range
            try:
                min_salary = int(op_json['estimatedSalary']['value']['minValue'])
                max_salary = int(op_json['estimatedSalary']['value']['maxValue'])
            except:
                min_salary = -1
                max_salary = -1
            #Unidad salarial
            try:
                unitText = op_json['estimatedSalary']['value']['unitText']
            except:
                unitText = None
            #salary_currency
            try:
                salary_currency = op_json['salaryCurrency']
            except:
                salary_currency = None
            #Job Posting validity date(Y-M-D)
            try:
                valid_date = op_json['validThrough']
            except:
                valid_date = None
            #Industry
            try:
                industry = op_json['industry']
            except:
                industry = None
            #Location
            try:
                city = op_json['jobLocation']['address']['addressLocality']
                state = op_json['jobLocation']['address']['addressRegion']
            except:
                city , state = None , None
            #Company
            try:
                company = op_json['hiringOrganization']['name']
            except:
                company = None
            #Let's get Description
            try:
                desc = soup.find(class_ = 'desc').text
            except:
                desc = None
            try:
                rating = soup.find('span' , {'class' : 'css-1pmc6te e11nt52q3'}).text.replace('★' , '')
            except:
                rating = None
            data_df = data_df.append({'Job_title' : title,
                                  'Company' : company,
                                  'State' : state,
                                  'City' : city,
                                  'Min_Salary':min_salary,
                                  'Max_Salary':max_salary,
                                  'unitText': unitText,
                                  'SalaryCurrency': salary_currency,
                                  'Job_Desc' : desc,
                                  'Industry':industry,
                                  'Rating':rating,
                                  'Date_Posted' : post_date,
                                  'Valid_until' : valid_date,
                                  'Job_Type' :job_type} , ignore_index = True)
        except:
            print(f'Trouble with job : {job+1}')
            pass
        #Para evitar problemas de bloqueo de la IP, vamos a dar 30 segundos de respiro por cada 50 trabajos:
        if((job+1)%50 == 0):
            print("Time to pause...")
            time.sleep(30)
            print("Continue:")
    print(f"Scraping Completed for {data_df.shape[0]} jobs")
    print(f"Time Required : {time.time() - start} seconds")
    print(f"Time Required : {(time.time() - start)/60} minutes")
    return data_df

Veamos un ejemplo:
Primero los enlaces de la primera página que se consigue al buscar "cientifico de datos" en glassdoor

In [12]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver' , options = options)
br = driver
works = ScrapeLink(driver, "https://www.glassdoor.com.mx/Empleo/méxico-cientifico-de-datos-empleos-SRCH_IL.0,6_IN169_KO7,26.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=cientifico%2520de%2520datos&typedLocation=M%25C3%25A9xico&context=Jobs&dropdown=0")

Scraping web page
Scraping of links is completed


Por ejemplo:

In [22]:
works[5]

'https://www.glassdoor.com.mx/partner/jobListing.htm?pos=112&ao=1136043&s=58&guid=00000183495d5170aff680ae6ad0e167&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_f56b3894&cb=1663383196245&jobListingId=1007921490231&jrtk=3-0-1gd4lqkcqi6hs801-1gd4lqkdck6fc800-e7e514a048c5fe9b-'

Después se obtiene el código fuente con toda la información del trabajo en formato JSON

In [24]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver' , options = options)
br = driver
br.get(works[5])
soup = BeautifulSoup(br.page_source , 'lxml')
j = None
try:
    #The rest of the data is available in scrip = 'application/ld+json' i.e in  json file
    json_file = soup.find('script' , {'type':'application/ld+json'})
    op_json = json.loads(str(json_file.text) , strict = False)
    j = op_json
except:
    desc = None

Por ejemplo:

In [25]:
j

{'@context': 'https://schema.org/',
 '@type': 'JobPosting',
 'title': 'CIENTIFICO DE DATOS (PYTHON)',
 'url': '',
 'datePosted': '2022-09-06T00:00:00',
 'employmentType': 'FULL_TIME',
 'validThrough': '2022-10-17',
 'hiringOrganization': {'@type': 'Organization',
  'name': 'SYCNOS SERVICIOS Y CONSULTORIA ESTRATEGICA SA DE CV',
  'logo': 'https://media.glassdoor.com/sql/2726346/sycnos-squarelogo-1634293377423.png',
  'sameAs': 'http://www.sycnos.com'},
 'jobLocation': {'@type': 'Place',
  'address': {'@type': 'PostalAddress',
   'addressLocality': 'Monterrey',
   'addressRegion': 'Nuevo León',
   'addressCountry': {'@type': 'Country', 'name': 'Mexico'}},
  'geo': {'@type': 'GeoCoordinates',
   'latitude': 25.666666,
   'longitude': -100.316666}},
 'description': '&lt;p&gt;Requisitos:&lt;/p&gt;&lt;ul&gt;&lt;li&gt;Analítica de datos, de procesos de negocio.&lt;/li&gt;&lt;li&gt;Generación de soluciones de analítica avanzada mediante técnicas y metodologías de ciencia de datos en el lenguaj

Los siguientes son tres ejemplos del procedimiento que se hizo para diferentes grupos de enlaces derivados de diferentes busquedas de conceptos (Analista de datos, Ingeniero de datos, Data Analyst, Machine Learning Engineer, Científico de datos, Artificial Intelligence Engineer, Data Scientist Jr, Data Scientist Junior, Data Engineering Junior, Analista en Estadística, etc.)

In [4]:
analistaEnEstadistica_links_glassdor = [
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=analista%2520estad%25C3%25ADstico&typedLocation=M%25C3%25A9xico&context=Jobs&dropdown=0",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP2.htm?includeNoSalaryJobs=true&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAeS55uMAMQEBAQcEbJYpl3PkK%2BJgWA4DaXlYcUu892A6cS0FhRoLLj3tcpeDHgIZl5R6kluyMu8AAA%3D%3D",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP3.htm?includeNoSalaryJobs=true&pgc=AB4AAoEAPAAAAAAAAAAAAAAAAeS55uMAUQECAQgaCgG7VQmkyWhps2ljyEq03EDW%2Bx3D%2F1HV%2FjGD2tNIs4CibBqM376TvgTyfcegpWGewPYkXmz0t8%2B1meIKelEay8stNygeFQllxL8OFgAA",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP4.htm?includeNoSalaryJobs=true&pgc=AB4AA4EAWgAAAAAAAAAAAAAAAeS55uMAYAECARImBg9tEMJVM5UZXk9Xw8K7FoAWTRSac1DaODWZoSeOwpsTwe9aTIa3TsMt4i86tq3wd6igEKNziP4oZBQzn58VJTpFYFBD2uBGi91kZoCZ1wv6RfR5Sr1cAUYZTgAA",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP5.htm?includeNoSalaryJobs=true&pgc=AB4ABIEAeAAAAAAAAAAAAAAAAeS55uMAZAEDARImDCwGA%2FSwn25z90v3l0lQ6Um6Ug7Ctz6ZtXo3z6VtHJLs4Vcm5qQyYS4IbgAE7hVTsLiqmeJqoid6oKR7JXbyJU7S%2FVTPIT%2Bqyl6V64ckcEwDz5bj0mI72svt%2BxR7M04AAA%3D%3D",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP6.htm?includeNoSalaryJobs=true&pgc=AB4ABYEAlgAAAAAAAAAAAAAAAeS55uMAYwECASJUBi84SRA%2FZP%2FlxVEVMqFoiwjcmPgeCyeZbV9SAHjCPGe44uZSZPZIVBGCjnGozl8tyyJqSmoqW5Q%2BNF19lnbQw1dtbtVTzHToJDdUeINLuglv4bPmx%2FM%2B%2BaYfzQY3ZgAA",
    "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-analista-estad%C3%ADstico-empleos-SRCH_IL.0,6_IN169_KO7,27_IP7.htm?includeNoSalaryJobs=true"
]
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver' , options = options)
#Obtenemos la lista de links de los trabajos de glassdor
#En esta, es donde se pueden eliminar los últimos 2 parámetros si tenemos menos de 6 enlaces
analistaEnEstadistica_links_JG = get_link_jobs(driver, analistaEnEstadistica_links_glassdor, 7, 10)

#Al final obtendremos el data frame
driver = webdriver.Chrome('chromedriver' , options = options)
analistaEnEstadistica_data_G = ScrapeData(driver, analistaEnEstadistica_links_JG)

analistaEnEstadistica_data_G = analistaEnEstadistica_data_G.drop_duplicates()
analistaEnEstadistica_data_G.reset_index(drop=True, inplace=True)

analistaEnEstadistica_data_G

Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de páginas con patrón
Scraping Page Number : 7
Scraping Page Number : 8
Scraping Page Number : 9
Scraping Page Number : 10
Scraping of links is completed
Número de links:  300
Gathering Information
Extracting information about job : 1
Extracting information about job : 2
Extracting information about job : 3
Extracting information about job : 4
Extracting information about job : 5
Extracting information about job : 6

,Job_title,Company,State,City,Min_Salary,Max_Salary,unitText,SalaryCurrency,Job_Desc,Industry,Rating,Date_Posted,Valid_until,Job_Type
0,CPFR,Comercializadora y Distribuidora de Vinos y Li...,None,None,360000,420000,MONTH,MXN,Descripción del empleoImportante empresa comer...,None,None,2022-08-17T00:00:00,2022-10-16,FULL_TIME
1,Analista de mercado´,Grupo ADO,None,None,207600,207600,MONTH,MXN,Somos personas que trabajan con y para las per...,None,None,2022-09-13T00:00:00,2022-10-16,FULL_TIME
2,ANALISTA RIESGOS JR AMEDIRH evaluatest,Amedirh Talento,None,None,284700,284700,MONTH,MXN,ACTIVIDADES\nMedición del riesgo de las operac...,None,None,2022-09-09T00:00:00,2022-10-16,OTHER
3,Becario Analista SEM,Sube Agencia Digital,None,None,-1,-1,None,MXN,"SUBE Agencia Digital es una empresa mexicana, ...",None,None,2022-09-15T00:00:00,2022-10-16,FULL_TIME
4,Analista de Datos,GRUPO CSI,Baja California,Tijuana,192000,240000,MONTH,MXN,Grupo CSI solicita coordinador de inteligencia...,None,None,2022-09-15T00:00:00,2022-10-16,FULL_TIME
5,Analista E-commerce,Hanesbrands,México,Tlalnepantla,204000,210000,MONTH,MXN,Analista E-Commerce Requisitos: Escolaridad: L...,Manufactura,None,2022-09-08T00:00:00,2022-10-16,FULL_TIME
6,Analista de Investigación y Desarrollo de Mercado,Ácidos y Solventes S.A de C.V,México,Tlalnepantla,144000,144000,MONTH,MXN,"FORMACIÓN: Lic. Mercadotecnia, Ingeniería Quím...",None,None,2022-09-02T00:00:00,2022-10-16,FULL_TIME
7,Analista de Datos,PSM Asesores,Nuevo León,San Pedro Garza García,180000,180000,MONTH,MXN,Consultora líder en Category Management en Méx...,None,None,2022-08-17T00:00:00,2022-10-16,FULL_TIME
8,Analista Deportivo - Hockey,InStat,México,Ciudad de México,-1,-1,None,MXN,InStat se ha establecido como líder del mercad...,None,None,2022-08-26T00:00:00,2022-10-16,FULL_TIME
9,Analista de datos (operaciones),ARIOAC,Jalisco,Zapopan,216000,276000,MONTH,MXN,Ofrecemos: Horario: Lunes a Viernes de 9:00 a ...,None,None,2022-09-08T00:00:00,2022-10-16,FULL_TIME


In [5]:
analistaEnEstadistica_data_G.to_csv("Analista_En_Estadistica.csv", index = False)

In [19]:
analistaDatosSR_links_glassdor = [
    "https://www.glassdoor.com/Job/mexico-analista-de-datos-sr-jobs-SRCH_IL.0,6_IN169_KO7,27.htm?clickSource=searchBox",
    "https://www.glassdoor.com/Job/mexico-analista-de-datos-sr-jobs-SRCH_IL.0,6_IN169_KO7,27_IP2.htm?includeNoSalaryJobs=true&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAeRxijEACwEAbRV65ahobYwaAAA%3D"
]

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver' , options = options)
#Obtenemos la lista de links de los trabajos de glassdor
#En esta, es donde se pueden eliminar los últimos 2 parámetros si tenemos menos de 6 enlaces
analistaDatosSR_links_JG = get_link_jobs(driver, analistaDatosSR_links_glassdor)

#Al final obtendremos el data frame
driver = webdriver.Chrome('chromedriver' , options = options)
analistaDatosSR_data_G = ScrapeData(driver, analistaDatosSR_links_JG)

analistaDatosSR_data_G = analistaDatosSR_data_G.drop_duplicates()
analistaDatosSR_data_G.reset_index(drop=True, inplace=True)

analistaDatosSR_data_G

Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Número de links:  50
Gathering Information
Extracting information about job : 1
Extracting information about job : 2
Extracting information about job : 3
Extracting information about job : 4
Extracting information about job : 5
Extracting information about job : 6
Extracting information about job : 7
Extracting information about job : 8
Extracting information about job : 9
Extracting information about job : 10
Extracting information about job : 11
Extracting information about job : 12
Extracting information about job : 13
Extracting information about job : 14
Extracting information about job : 15
Extracting information about job : 16
Extracting information about job : 17
Extracting information about job : 18
Extracting information about job : 19
Extracting information about job : 20
Extracting informati

,Job_title,Company,State,City,Min_Salary,Max_Salary,unitText,SalaryCurrency,Job_Desc,Industry,Rating,Date_Posted,Valid_until,Job_Type
0,Analista de Datos SR,DISH MEXICO,None,None,221988,221988,MONTH,MXN,Esta vacante viene de la bolsa de empleo Talen...,Telecomunicaciones,None,2022-09-15T00:00:00,2022-10-16,OTHER
1,Analista Sr Finanzas,Aon,None,None,-1,-1,None,MXN,¿Te gustaría unirte a un equipo diverso e incl...,Seguros,None,2022-08-20T00:00:00,2022-10-16,OTHER
2,Analista Sr. Contable,MMC Corporate,México,Ciudad de México,-1,-1,None,MXN,Marsh está buscando candidatos para la posició...,Seguros,None,2022-09-15T00:00:00,2022-10-16,FULL_TIME
3,"Sr. Analyst, Training & Content Data",Herbalife,México,Ciudad de México,-1,-1,None,MXN,DescripciÃ³n:\n\nRESPONSABILIDADES DE LA POSIC...,Servicios personales al consumidor,None,2022-09-14T00:00:00,2022-10-16,FULL_TIME
4,Data Engineer SAS Híbrido CDMX,Reclutamiento IT,None,None,660000,660000,MONTH,MXN,Esta vacante viene de la bolsa de empleo Talen...,None,None,2022-09-13T00:00:00,2022-10-16,FULL_TIME
5,Desarrollador PHP Magento Home Office,Training Talent,None,None,600000,600000,MONTH,MXN,Esta vacante viene de la bolsa de empleo Talen...,None,None,2022-09-14T00:00:00,2022-10-16,FULL_TIME
6,Ingeniero de Datos SAS Híbrido CDMX,Training Talent,None,None,660000,660000,MONTH,MXN,Esta vacante viene de la bolsa de empleo Talen...,None,None,2022-09-13T00:00:00,2022-10-16,FULL_TIME
7,MCA Tester,Citi,México,Ciudad de México,-1,-1,None,MXN,El Analista II de Riesgo Operativo es una func...,Finanzas,None,2022-09-13T00:00:00,2022-10-16,FULL_TIME
8,Analista de Datos,Proyecto de Repositorio,México,Ciudad de México,300000,396000,MONTH,MXN,Analista de Datos – SrConsultor de nivel avanz...,None,None,2022-08-28T00:00:00,2022-10-16,FULL_TIME
9,Líder de Pruebas QA Software CDMX Híbrido,Reclutamiento IT,None,None,360000,360000,MONTH,MXN,Esta vacante viene de la bolsa de empleo Talen...,None,None,2022-09-12T00:00:00,2022-10-16,FULL_TIME


In [20]:
analistaDatosSR_data_G.to_csv("analista_Datos_SR.csv", index = False)

In [28]:
cientificoDatosSenior_links_JG = get_link_jobs(driver, "https://www.glassdoor.com.mx/Empleo/m%C3%A9xico-cientifico-de-datos-senior-empleos-SRCH_IL.0,6_IN169_KO7,33.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Cientifico%2520de%2520datos%2520senior&typedLocation=M%25C3%25A9xico&context=Jobs&dropdown=0")

driver = webdriver.Chrome('chromedriver' , options = options)
cientificoDatosSenior_data_G = ScrapeData(driver, cientificoDatosSenior_links_JG)

cientificoDatosSenior_data_G = cientificoDatosSenior_data_G.drop_duplicates()
cientificoDatosSenior_data_G.reset_index(drop=True, inplace=True)

cientificoDatosSenior_data_G

cientificoDatosSenior_data_G.to_csv("cientifico_Datos_Senior.csv", index = False)

Comienza Web Scrapping de página individual
Scraping web page
Scraping of links is completed
Número de links:  7
Gathering Information
Extracting information about job : 1
Extracting information about job : 2
Extracting information about job : 3
Extracting information about job : 4
Extracting information about job : 5
Extracting information about job : 6
Extracting information about job : 7
Scraping Completed for 7 jobs
Time Required : 40.54603385925293 seconds
Time Required : 0.675768764813741 minutes
